In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv("/content/drive/MyDrive/outlier/이재명.csv")
data.head()


,tweet,date,retweets_count,link
0,윤석열 vs 이재명 인성 논란 이 영상 하나로 종결 짓습니다 심각한 언론 형평성 4...,2021-11-22,0,https://twitter.com/SungJun_777/status/1462764...
1,jtkim1013 좌파들아 이재명을 구속해 그래야 윤돌이도 구속하지,2021-11-22,0,https://twitter.com/sosara609/status/146276435...
2,지도부 빼고 취준생 앉히고 이재명 부동산 대장동 반성 사과 B98BA8fnHt,2021-11-22,0,https://twitter.com/love2poem/status/146276434...
3,TheMinjoo_Kr 국힘당 보다 수준 떨어지는 만주당 상태 북한까지 이재명의 ...,2021-11-22,0,https://twitter.com/lanotte702/status/14627642...
4,진성준 아주 무례하고 개매너인 거 이재명을 위해 온 몸을 바쳐 막아가며 소리지르고 ...,2021-11-22,1,https://twitter.com/digidigi0208/status/146276...


In [4]:
range(len(data["tweet"]))

range(0, 59880)

# 정수 인코딩과 단어 집합 만들기

In [5]:
!pip install konlpy
from konlpy.tag import Kkma  
kkma = Kkma()
tokenized_doc=[]
for n in range(len(data["tweet"])):
    text = data["tweet"][n]
    tokenized_doc.append(list(kkma.nouns(str(text))))
print(tokenized_doc[:5])

     |████████████████████████████████| 19.4 MB 12.3 MB/s 
     |████████████████████████████████| 448 kB 98.1 MB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
[['윤', '열', '이재명', '인성', '논란', '이', '영상', '하나', '종결', '심각', '언론', '형평', '40', '1'], ['1013', '좌파', '이재명', '구속', '윤', '윤돌이', '돌이'], ['지도부', '취', '생', '이재명', '부동산', '대장', '대장동', '동', '반성', '사과', '98', '8'], ['국', '국힘당', '힘', '당', '수준', '만주', '상태', '북한', '이재', '이재명의', '명의', '내가', '진동', '민주당', '이재명', '전과', '전과4범', '4', '범'], ['진성', '무례', '개', '개매너', '매너', '거', '이재명', '몸', '난리', '양아치', '짓거리', '하다', '대장', '대장동이', '동이', '관련', '발악', '그때', '의심']]


In [6]:
print(tokenized_doc[2])

['지도부', '취', '생', '이재명', '부동산', '대장', '대장동', '동', '반성', '사과', '98', '8']


In [7]:
kkma.nouns(data["tweet"][2])

['지도부', '취', '생', '이재명', '부동산', '대장', '대장동', '동', '반성', '사과', '98', '8']

이제 각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수를 기록. 여기서는 각 단어를 (word_id, word_frequency)의 형태로 바꾸고자 합니다. word_id는 단어가 정수 인코딩된 값이고, word_frequency는 해당 뉴스에서의 해당 단어의 빈도수를 의미합니다. 이는 gensim의 corpora.Dictionary()를 사용하여 손쉽게 구할 수 있습니다. 전체 뉴스에 대해서 정수 인코딩을 수행하고, 세번째 댓글을 추출합니다.

In [9]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[2])#세번째 댓글 출력

[(9, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


총 학습된 단어의 개수를 확인

In [13]:
len(dictionary) #61145개 단어가 학습

61145

#  LDA 모델을 훈련

각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줍니다. 또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 20개의 토픽은 0부터 19까지의 번호가 할당되어져 있습니다. passes는 알고리즘의 동작 횟수를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 됩니다. 여기서는 총 15회를 수행하였습니다. 여기서는 num_words=4로 총 4개의 단어만 출력하도록 하였습니다. 만약 10개의 단어를 출력하고 싶다면 아래의 코드를 수행하면 됩니다.

### 토픽번호별 키워드중요도

In [23]:
import gensim
NUM_TOPICS = 20 #20개의 토픽(카테고리), k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

(0, '0.083*"이재명" + 0.030*"거" + 0.021*"사람" + 0.019*"말" + 0.013*"레" + 0.013*"지지" + 0.013*"생각"')
(1, '0.030*"이재명" + 0.028*"4" + 0.025*"전과" + 0.024*"범" + 0.015*"탓" + 0.013*"남편" + 0.012*"상처"')
(2, '0.045*"이재명" + 0.019*"진중" + 0.017*"욕" + 0.017*"비하" + 0.015*"쌍" + 0.015*"119" + 0.013*"방문"')
(3, '0.048*"이재명" + 0.047*"교체" + 0.044*"낙" + 0.042*"후보" + 0.032*"연" + 0.030*"안" + 0.029*"후보교체"')
(4, '0.106*"윤" + 0.072*"이재명" + 0.071*"열" + 0.024*"5" + 0.023*"1" + 0.022*"2" + 0.021*"4"')
(5, '0.057*"이재명" + 0.021*"김" + 0.018*"글" + 0.016*"언론" + 0.013*"가짜" + 0.011*"후보" + 0.011*"씨"')
(6, '0.064*"동" + 0.062*"대장" + 0.054*"대장동" + 0.039*"이재명" + 0.032*"수사" + 0.030*"특검" + 0.021*"검찰"')
(7, '0.030*"국" + 0.029*"힘" + 0.028*"이재명" + 0.025*"민국" + 0.025*"대한" + 0.025*"대한민국" + 0.019*"당"')
(8, '0.040*"부산" + 0.024*"이재명" + 0.023*"이재" + 0.019*"발언" + 0.018*"명의" + 0.017*"민주당" + 0.017*"이재명의"')
(9, '0.041*"이재명" + 0.017*"우리" + 0.016*"후보" + 0.012*"대통령" + 0.011*"언론" + 0.011*"문프" + 0.009*"국민"')
(10, '0.029*"도" + 0.028*"경기" + 0.021*"이재명" +

In [32]:
print(ldamodel.print_topics())

[(0, '0.083*"이재명" + 0.030*"거" + 0.021*"사람" + 0.019*"말" + 0.013*"레" + 0.013*"지지" + 0.013*"생각" + 0.012*"기" + 0.011*"놈" + 0.011*"민주당"'), (1, '0.030*"이재명" + 0.028*"4" + 0.025*"전과" + 0.024*"범" + 0.015*"탓" + 0.013*"남편" + 0.012*"상처" + 0.011*"남" + 0.011*"전과4범" + 0.011*"수"'), (2, '0.045*"이재명" + 0.019*"진중" + 0.017*"욕" + 0.017*"비하" + 0.015*"쌍" + 0.015*"119" + 0.013*"방문" + 0.013*"너" + 0.009*"신고" + 0.009*"수"'), (3, '0.048*"이재명" + 0.047*"교체" + 0.044*"낙" + 0.042*"후보" + 0.032*"연" + 0.030*"안" + 0.029*"후보교체" + 0.028*"절대" + 0.026*"낙연" + 0.019*"길"'), (4, '0.106*"윤" + 0.072*"이재명" + 0.071*"열" + 0.024*"5" + 0.023*"1" + 0.022*"2" + 0.021*"4" + 0.021*"6" + 0.020*"3" + 0.019*"7"'), (5, '0.057*"이재명" + 0.021*"김" + 0.018*"글" + 0.016*"언론" + 0.013*"가짜" + 0.011*"후보" + 0.011*"씨" + 0.010*"뉴스" + 0.010*"가짜뉴스" + 0.009*"댓"'), (6, '0.064*"동" + 0.062*"대장" + 0.054*"대장동" + 0.039*"이재명" + 0.032*"수사" + 0.030*"특검" + 0.021*"검찰" + 0.019*"의혹" + 0.019*"게이트" + 0.013*"비리"'), (7, '0.030*"국" + 0.029*"힘" + 0.028*"이재명" + 0.025*"민국" + 0.025*

# 문서 관점에서 topic과 document 알기(문서별 토픽 분포 보기)

토픽별 단어 분포는 위에서 확인하였음. 문서별 토픽 분포에 대해 확인을 위해 이미 훈련된 LDA모델인 ldamodel[]에 전체 데이터가 정수 인코딩 된 결과를 넣은 후 확인 가능. 

상위 5개의 문서에 대해서만 토픽분포 확인

In [33]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(0, 0.18097003), (3, 0.07466822), (4, 0.46550915), (5, 0.14208873), (19, 0.08676386)]
1 번째 문서의 topic 비율은 [(0, 0.15823852), (3, 0.43966922), (4, 0.15392137), (5, 0.14817093)]
2 번째 문서의 topic 비율은 [(2, 0.08831254), (6, 0.34434733), (15, 0.16794156), (17, 0.33786014)]
3 번째 문서의 topic 비율은 [(1, 0.29843616), (7, 0.30338165), (8, 0.17939803), (17, 0.17878412)]
4 번째 문서의 topic 비율은 [(0, 0.21850252), (1, 0.2193074), (4, 0.055504344), (6, 0.10862458), (7, 0.10900551), (15, 0.1082441), (16, 0.06431694), (17, 0.08649461)]


위의 출력 결과에서 (숫자, 확률)은 토픽 번호와 해당토픽이 해당문서에서 차지하는 분포도를 의미.예를 들어 2번째 문서의 토픽 비율에서 (5, 0.16109002)은 5번 토픽이 16%의 분포도를 가지는 것을 의미

좀 더 깔끔한 형태의 데이터프레임 형식으로 출력

In [34]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [35]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10] #상위 10개 댓글별 비중이 높은 토픽은 무엇이고 얼만큼 차지하는지

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,4.0,0.4654,"[(0, 0.18052846), (3, 0.07467082), (4, 0.46535..."
1,1,3.0,0.4397,"[(0, 0.15818928), (3, 0.4396817), (4, 0.153949..."
2,2,6.0,0.3443,"[(2, 0.08831719), (6, 0.34426385), (15, 0.1679..."
3,3,7.0,0.3034,"[(1, 0.29843968), (7, 0.30338684), (8, 0.17939..."
4,4,1.0,0.2193,"[(0, 0.21849747), (1, 0.21930802), (4, 0.05550..."
5,5,16.0,0.2550,"[(3, 0.03375878), (5, 0.07629722), (6, 0.21021..."
6,6,15.0,0.6833,"[(0, 0.016666666), (1, 0.016666666), (2, 0.016..."
7,7,10.0,0.4613,"[(0, 0.3587143), (1, 0.010000001), (2, 0.01000..."
8,8,2.0,0.4560,"[(2, 0.4559845), (3, 0.0935964), (5, 0.2886062..."
9,9,11.0,0.7545,"[(11, 0.75445396), (15, 0.13304609)]"


# 단어 관점에서topic과 documents를 알기


In [39]:
keyword="기본소득"
id_list=[]
for i in range(len(data)):
  if keyword in tokenized_doc[i]:
    print("해당 단어가 들어간 댓글:",i,"번 댓글",":",data['tweet'][i])
    for y in range(len(tokenized_doc[i])):
      if tokenized_doc[i][y]==keyword:
        key_id=corpus[i][y]
        print("해당단어의 아이디는",key_id)
        id_list.append(key_id[0])

해당 단어가 들어간 댓글: 138 번 댓글 : 이재명에 대해 배웁시다 전과4범 김부선 총각행세 불륜 형보수지 사기꾼 음주운전 대장동 화천대유타운 유동규 정진상 백종선 신건수 나눔기획 만달러공작금 여론조작 혜경궁 0189대포폰 조폭친구 사사오입 기본소득 지역화폐 쿠팡 물류센터 화재 떡볶이 먹방 인성파탄 개자지 귀태
해당단어의 아이디는 (1046, 1)
해당 단어가 들어간 댓글: 160 번 댓글 :  TheMinjoo_Kr Jaemyung_Lee 인구의 급격한 감소는 나라가 망하는 지름길 입니다 해결책은 기본소득외는 대안이 없습니다 EDQsaNTpJi
해당단어의 아이디는 (1220, 1)
해당 단어가 들어간 댓글: 414 번 댓글 : 이재명 지지자들이 이상이 교수님을 협박하고 괴롭히는데 이것은 범죄입니다 범죄 공갈 협박 등 임을 인지 못하는 과실은 본인에게 있고요 이재명 지지자들은 스스로를 진흙탕에 처박지 말고 기본소득이든 보편적 복지국가이든 공부를 한 후에 얘기 하세요 사람들이 무식하다고 손가락질 하잖아요 
해당단어의 아이디는 (854, 1)
해당 단어가 들어간 댓글: 2185 번 댓글 :  Jaemyung_Lee 고용보험을 기본소득 재원으로 사용하면 어떨까요 
해당단어의 아이디는 (4213, 1)
해당 단어가 들어간 댓글: 2194 번 댓글 : 오세훈 이재명 기본소득 갈피 못잡아 직격탄 eTGdhUKK55
해당단어의 아이디는 (2034, 1)
해당 단어가 들어간 댓글: 2411 번 댓글 : 야 냅둬 그래야 빨리 망해 與 김영춘 이재명 기본소득 국토보유세보다 혁신에 집중해야 다음뉴스 PWm6AVztPB
해당단어의 아이디는 (854, 1)
해당 단어가 들어간 댓글: 2450 번 댓글 :  안심소득은 정부 승인을 얻어 당장 내년부터 시범사업을 실시할 수 있게 됐다 며 같은 당내에서도 동의를 못받아 갈피를 잡지 못하는 기본소득과는 많이 다르다 오세훈 안심소득은 이재명 기본소득과 실현 가능성 달라 출처 뉴스1 네이버 뉴스 9vGaC4VFDl
해당단어의 아이디는 (203

In [41]:
# 기본소득에 관련된 토픽(+토픽에 영향주는 키워드) 찾기
term_topic_dist = ldamodel.get_term_topics(33, minimum_probability=0)
sorted_term_topic = sorted(term_topic_dist, key=lambda x:x[1], reverse=True)
sorted_term_topic

[(0, 0.005007456), (5, 0.0016459193), (13, 0.0015812635), (1, 0.0012454775)]